In [ ]:
from model_connection import *
model = get_model("qwen2.5-coder-14b-instruct")

In [3]:
!pip install 'smolagents[litellm]' plotly geopandas shapely kaleido -q

In [4]:
import math
from typing import Optional, Tuple

from smolagents import tool


@tool
def calculate_cargo_travel_time(
    origin_coords: Tuple[float, float],
    destination_coords: Tuple[float, float],
    cruising_speed_kmh: Optional[float] = 750.0,  # Average speed for cargo planes
) -> float:
    """
    Calculate the travel time for a cargo plane between two points on Earth using great-circle distance.

    Args:
        origin_coords: Tuple of (latitude, longitude) for the starting point
        destination_coords: Tuple of (latitude, longitude) for the destination
        cruising_speed_kmh: Optional cruising speed in km/h (defaults to 750 km/h for typical cargo planes)

    Returns:
        float: The estimated travel time in hours

    Example:
        >>> # Chicago (41.8781° N, 87.6298° W) to Sydney (33.8688° S, 151.2093° E)
        >>> result = calculate_cargo_travel_time((41.8781, -87.6298), (-33.8688, 151.2093))
    """

    def to_radians(degrees: float) -> float:
        return degrees * (math.pi / 180)

    # Extract coordinates
    lat1, lon1 = map(to_radians, origin_coords)
    lat2, lon2 = map(to_radians, destination_coords)

    # Earth's radius in kilometers
    EARTH_RADIUS_KM = 6371.0

    # Calculate great-circle distance using the haversine formula
    dlon = lon2 - lon1
    dlat = lat2 - lat1

    a = (
        math.sin(dlat / 2) ** 2
        + math.cos(lat1) * math.cos(lat2) * math.sin(dlon / 2) ** 2
    )
    c = 2 * math.asin(math.sqrt(a))
    distance = EARTH_RADIUS_KM * c

    # Add 10% to account for non-direct routes and air traffic controls
    actual_distance = distance * 1.1

    # Calculate flight time
    # Add 1 hour for takeoff and landing procedures
    flight_time = (actual_distance / cruising_speed_kmh) + 1.0

    # Format the results
    return round(flight_time, 2)


print(calculate_cargo_travel_time((41.8781, -87.6298), (-33.8688, 151.2093)))

22.82


In [6]:
import os
from PIL import Image
from smolagents import CodeAgent, GoogleSearchTool, InferenceClientModel, VisitWebpageTool

In [7]:
task = """Find all Batman filming locations in the world, calculate the time to transfer via cargo plane to here (we're in Gotham, 40.7128° N, 74.0060° W), and return them to me as a pandas dataframe.
Also give me some supercar factories with the same cargo plane transfer time."""

In [29]:
SERPAPI_API_KEY = os.environ["SERPAPI_API_KEY"]

In [35]:
agent = CodeAgent(
    model=model,
    tools=[GoogleSearchTool(), VisitWebpageTool(), calculate_cargo_travel_time],
    additional_authorized_imports=["pandas"],
    max_steps=5,
)

In [36]:
result = agent.run(task)

╭──────────────────────────────────────────────────── New run ────────────────────────────────────────────────────╮
│                                                                                                                 │
│ Find all Batman filming locations in the world, calculate the time to transfer via cargo plane to here (we're   │
│ in Gotham, 40.7128° N, 74.0060° W), and return them to me as a pandas dataframe.                                │
│ Also give me some supercar factories with the same cargo plane transfer time.                                   │
│                                                                                                                 │
╰─ OpenAIServerModel - meta-llama-3.1-8b-instruct ────────────────────────────────────────────────────────────────╯

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 1 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

─ Executing parsed code: ──────────────────────────────────────────────────────────────────────────────────────── 
  filming_locations = web_search(query="Batman filming locations")                                                 
  print(filming_locations)                                                                                         
 ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────

Execution logs:
## Search Results
0. [The Batman (2022) - Filming & production](https://www.imdb.com/title/tt1877830/locations/)
Source: IMDb

Filming locations: Necropolis Cemetery, Glasgow, Scotland, UK (Batman and Selina leaving the cemetery) Helpful•87 1
St. George's Hall, Liverpool, England, UK

1. [The Batman | Film Locations](https://movie-locations.com/movies/b/The-Batman-2022-2.php)
Source: The Worldwide Guide To Movie Locations

The Batman filming location: St George's Hall, Liverpool · The Batman filming location: Royal Liver Building, Pier 
Head, Liverpool · The Batman filming location: ...

2. [Batman (1989) - Filming & production](https://www.imdb.com/title/tt0096895/locations/)
Source: IMDb

Filming locations ; Knebworth House, Knebworth, Hertfordshire, England, UK. (Wayne Manor; exterior) · 21 ; Acton 
Lane Power Station, Acton Lane, Acton, London, ...

3. [Where was The Batman filmed? ALL the Filming Locations 
...](https://www.atlasofwonders.com/2022/04/where-was-the-batman-filmed.html)
Source: Atlas of Wonders

The Batman was primarily filmed in the United Kingdom. Most of the recognizable buildings of this new version of 
Gotham City are located in Liverpool.

4. [12 Batman Movie Locations You Can 
Visit!](https://www.travelandleisureasia.com/sea/destinations/batman-movie-locations-you-can-visit/)
Date published: Jan 10, 2023
Source: Travel and Leisure Asia

Most of the filming of Batman movies is done in the Warner Bros studios and across the US, including New York and 
Pittsburgh. You will notice ...

5. [Batman Movies and Filming Locations in Los 
Angeles](https://www.traveltodayla.com/post/batman-movies-in-la?srsltid=AfmBOoq6o2DXRu9t1X-HpjJxC8WojsXuvPXrVC4n3m4
PYIbMh2sBBCaN)
Date published: Apr 20, 2023
Source: LA Today

Batman Movies and Filming Locations in LA · Cliffs of Marineland · Kirkeby Center · Gateway East Building · Wayne 
Manor · Bronson Caves · Batman ...

6. [Where was Batman Begins filmed?](https://giggster.com/guide/movie-location/where-was-batman-begins-filmed)
Source: Giggster

Batman Begins was filmed in various locations across the globe, with most scenes shot in the United Kingdom, 
specifically in London, Buckinghamshire, and Essex.

7. [The Batman (film)](https://en.wikipedia.org/wiki/The_Batman_(film))
Source: Wikipedia

Pattinson was cast in May 2019, with further casting later that year. Filming began in the United Kingdom in 
January 2020 but was halted by the COVID-19 ...

8. [The Dark Knight | 2008](https://movie-locations.com/movies/d/Dark-Knight.php)
Source: The Worldwide Guide To Movie Locations

Discover where The Dark Knight (2008) was filmed around Chicago, as well as in London and Bedfordshire in the UK, 
and briefly in Hong Kong.

9. [Locations where The Batman movie starring Robert 
...](https://www.whtimes.co.uk/news/22344457.revealed-locations-batman-movie-starring-robert-pattinson-filmed/)
Date published: Mar 3, 2022
Source: Welwyn Hatfield Times

As well as being filmed on location in Chicago, America, and London, Liverpool and Glasgow in the UK, The Batman 
was also shot on sets built at ...

Out: None

[Step 1: Duration 15.55 seconds| Input tokens: 2,210 | Output tokens: 61]

...

─ Executing parsed code: ──────────────────────────────────────────────────────────────────────────────────────── 
  batman_locations = web_search(query="Batman filming locations")                                                  
  print(batman_locations)                                                                                          
 ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────

Execution logs:
## Search Results
0. [The Batman (2022) - Filming & production](https://www.imdb.com/title/tt1877830/locations/)
Source: IMDb

Filming locations: Necropolis Cemetery, Glasgow, Scotland, UK (Batman and Selina leaving the cemetery) Helpful•87 1
St. George's Hall, Liverpool, England, UK

1. [The Batman | Film Locations](https://movie-locations.com/movies/b/The-Batman-2022-2.php)
Source: The Worldwide Guide To Movie Locations

The Batman filming location: St George's Hall, Liverpool · The Batman filming location: Royal Liver Building, Pier 
Head, Liverpool · The Batman filming location: ...

2. [Batman (1989) - Filming & production](https://www.imdb.com/title/tt0096895/locations/)
Source: IMDb

Filming locations ; Knebworth House, Knebworth, Hertfordshire, England, UK. (Wayne Manor; exterior) · 21 ; Acton 
Lane Power Station, Acton Lane, Acton, London, ...

3. [Where was The Batman filmed? ALL the Filming Locations 
...](https://www.atlasofwonders.com/2022/04/where-was-the-batman-filmed.html)
Source: Atlas of Wonders

The Batman was primarily filmed in the United Kingdom. Most of the recognizable buildings of this new version of 
Gotham City are located in Liverpool.

4. [12 Batman Movie Locations You Can 
Visit!](https://www.travelandleisureasia.com/sea/destinations/batman-movie-locations-you-can-visit/)
Date published: Jan 10, 2023
Source: Travel and Leisure Asia

Most of the filming of Batman movies is done in the Warner Bros studios and across the US, including New York and 
Pittsburgh. You will notice ...

5. [Batman Movies and Filming Locations in Los 
Angeles](https://www.traveltodayla.com/post/batman-movies-in-la?srsltid=AfmBOoq6o2DXRu9t1X-HpjJxC8WojsXuvPXrVC4n3m4
PYIbMh2sBBCaN)
Date published: Apr 20, 2023
Source: LA Today

Batman Movies and Filming Locations in LA · Cliffs of Marineland · Kirkeby Center · Gateway East Building · Wayne 
Manor · Bronson Caves · Batman ...

6. [Where was Batman Begins filmed?](https://giggster.com/guide/movie-location/where-was-batman-begins-filmed)
Source: Giggster

Batman Begins was filmed in various locations across the globe, with most scenes shot in the United Kingdom, 
specifically in London, Buckinghamshire, and Essex.

7. [The Batman (film)](https://en.wikipedia.org/wiki/The_Batman_(film))
Source: Wikipedia

Pattinson was cast in May 2019, with further casting later that year. Filming began in the United Kingdom in 
January 2020 but was halted by the COVID-19 ...

8. [The Dark Knight | 2008](https://movie-locations.com/movies/d/Dark-Knight.php)
Source: The Worldwide Guide To Movie Locations

Discover where The Dark Knight (2008) was filmed around Chicago, as well as in London and Bedfordshire in the UK, 
and briefly in Hong Kong.

9. [Locations where The Batman movie starring Robert 
...](https://www.whtimes.co.uk/news/22344457.revealed-locations-batman-movie-starring-robert-pattinson-filmed/)
Date published: Mar 3, 2022
Source: Welwyn Hatfield Times

As well as being filmed on location in Chicago, America, and London, Liverpool and Glasgow in the UK, The Batman 
was also shot on sets built at ...

Out: None

[Step 5: Duration 3.44 seconds| Input tokens: 14,456 | Output tokens: 333]

Reached max steps.

[Step 6: Duration 50.71 seconds| Input tokens: 17,770 | Output tokens: 805]

In [39]:
!pip install markdownify requests -q

In [38]:
agent.planning_interval = 4

detailed_report = agent.run(f"""
You're an expert analyst. You make comprehensive reports after visiting many websites.
Don't hesitate to search for many queries at once in a for loop.
For each data point that you find, visit the source url to confirm numbers.

{task}
""")

print(detailed_report)

╭──────────────────────────────────────────────────── New run ────────────────────────────────────────────────────╮
│                                                                                                                 │
│ You're an expert analyst. You make comprehensive reports after visiting many websites.                          │
│ Don't hesitate to search for many queries at once in a for loop.                                                │
│ For each data point that you find, visit the source url to confirm numbers.                                     │
│                                                                                                                 │
│ Find all Batman filming locations in the world, calculate the time to transfer via cargo plane to here (we're   │
│ in Gotham, 40.7128° N, 74.0060° W), and return them to me as a pandas dataframe.                                │
│ Also give me some supercar factories with the same cargo plane transfer time.                                   │
│                                                                                                                 │
╰─ OpenAIServerModel - meta-llama-3.1-8b-instruct ────────────────────────────────────────────────────────────────╯

────────────────────────────────────────────────── Initial plan ───────────────────────────────────────────────────
Here are the facts I know and the plan of action that I will follow to solve the task:
```
### 1. Facts Survey

#### 1.1. Facts given in the task
- None

#### 1.2. Facts to look up
- Batman filming locations: 
    - Where: website(s) with filming location information for Batman movies/TV shows (e.g., Wikipedia, IMDB)
- Cruising speed of typical cargo planes:
    - Where: aviation websites or Wikipedia article on cargo planes
- Supercar factories:
    - Where: list of supercar manufacturers and their locations

#### 1.3. Facts to derive
- Location coordinates of Batman filming locations (latitude, longitude):
    - Derive from website(s) with filming location information for Batman movies/TV shows
- Time to transfer via cargo plane between each filming location and Gotham:
    - Calculate using the cruising speed of typical cargo planes and the great-circle distance formula

### 2. Plan

1. **Find Batman Filming Locations**
    - Perform web search for "Batman filming locations"
    - Visit top search results and extract relevant information
    - Confirm extracted data points by visiting source URLs

2. **Extract Location Coordinates of Batman Filming Locations**
    - For each confirmed data point, perform web search for the location's latitude and longitude
    - Extract coordinates from search results

3. **Calculate Cargo Plane Travel Time between Gotham and Each Filming Location**
    - Use `calculate_cargo_travel_time` function with origin at Gotham (40.7128° N, 74.0060° W) and destination at 
each filming location
    - Store travel times in a pandas dataframe along with the corresponding filming locations

4. **Find Supercar Factories within Cargo Plane Transfer Time**
    - Perform web search for "supercar factory locations"
    - Extract relevant information from top search results
    - Use `calculate_cargo_travel_time` function to calculate cargo plane travel time between each supercar factory
and Gotham
    - Store factories with transfer times less than or equal to the maximum filming location transfer time

5. **Combine Filming Locations and Supercar Factories into a Single Pandas DataFrame**
    - Merge two dataframes: one for Batman filming locations and their corresponding cargo plane travel times, and 
another for supercar factories within the same travel time

6. **Finalize Answer**
    - Use `final_answer` function to provide the combined dataframe as the final answer.


```

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 1 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

─ Executing parsed code: ──────────────────────────────────────────────────────────────────────────────────────── 
  batman_filming_locations = web_search("Batman filming locations")                                                
  print(batman_filming_locations)                                                                                  
 ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────

Execution logs:
## Search Results
0. [The Batman (2022) - Filming & production](https://www.imdb.com/title/tt1877830/locations/)
Source: IMDb

Filming locations: Necropolis Cemetery, Glasgow, Scotland, UK (Batman and Selina leaving the cemetery) Helpful•87 1
St. George's Hall, Liverpool, England, UK

1. [The Batman | Film Locations](https://movie-locations.com/movies/b/The-Batman-2022-2.php)
Source: The Worldwide Guide To Movie Locations

The Batman filming location: St George's Hall, Liverpool · The Batman filming location: Royal Liver Building, Pier 
Head, Liverpool · The Batman filming location: ...

2. [Batman (1989) - Filming & production](https://www.imdb.com/title/tt0096895/locations/)
Source: IMDb

Filming locations ; Knebworth House, Knebworth, Hertfordshire, England, UK. (Wayne Manor; exterior) · 21 ; Acton 
Lane Power Station, Acton Lane, Acton, London, ...

3. [Where was The Batman filmed? ALL the Filming Locations 
...](https://www.atlasofwonders.com/2022/04/where-was-the-batman-filmed.html)
Source: Atlas of Wonders

The Batman was primarily filmed in the United Kingdom. Most of the recognizable buildings of this new version of 
Gotham City are located in Liverpool.

4. [12 Batman Movie Locations You Can 
Visit!](https://www.travelandleisureasia.com/sea/destinations/batman-movie-locations-you-can-visit/)
Date published: Jan 10, 2023
Source: Travel and Leisure Asia

Most of the filming of Batman movies is done in the Warner Bros studios and across the US, including New York and 
Pittsburgh. You will notice ...

5. [Batman Movies and Filming Locations in Los 
Angeles](https://www.traveltodayla.com/post/batman-movies-in-la?srsltid=AfmBOoq6o2DXRu9t1X-HpjJxC8WojsXuvPXrVC4n3m4
PYIbMh2sBBCaN)
Date published: Apr 20, 2023
Source: LA Today

Batman Movies and Filming Locations in LA · Cliffs of Marineland · Kirkeby Center · Gateway East Building · Wayne 
Manor · Bronson Caves · Batman ...

6. [Where was Batman Begins filmed?](https://giggster.com/guide/movie-location/where-was-batman-begins-filmed)
Source: Giggster

Batman Begins was filmed in various locations across the globe, with most scenes shot in the United Kingdom, 
specifically in London, Buckinghamshire, and Essex.

7. [The Batman (film)](https://en.wikipedia.org/wiki/The_Batman_(film))
Source: Wikipedia

Pattinson was cast in May 2019, with further casting later that year. Filming began in the United Kingdom in 
January 2020 but was halted by the COVID-19 ...

8. [The Dark Knight | 2008](https://movie-locations.com/movies/d/Dark-Knight.php)
Source: The Worldwide Guide To Movie Locations

Discover where The Dark Knight (2008) was filmed around Chicago, as well as in London and Bedfordshire in the UK, 
and briefly in Hong Kong.

9. [Locations where The Batman movie starring Robert 
...](https://www.whtimes.co.uk/news/22344457.revealed-locations-batman-movie-starring-robert-pattinson-filmed/)
Date published: Mar 3, 2022
Source: Welwyn Hatfield Times

As well as being filmed on location in Chicago, America, and London, Liverpool and Glasgow in the UK, The Batman 
was also shot on sets built at ...

Out: None

...

─ Executing parsed code: ──────────────────────────────────────────────────────────────────────────────────────── 
  import pandas as pd                                                                                              
                                                                                                                   
  # First, let's search for all Batman filming locations                                                           
  batman_locations = []                                                                                            
  for query in ["Batman filming locations", "The Dark Knight filming locations", "Christopher Nolan filming        
  locations"]:                                                                                                     
      print(f"Searching for {query}...")                                                                           
      results = web_search(query=query)                                                                            
      for result in results:                                                                                       
          batman_locations.append({"Location": result, "Source URL": f"{result}"})                                 
                                                                                                                   
  # Now let's calculate the distance from Gotham to each location                                                  
  from math import radians, sin, cos, sqrt, atan2                                                                  
                                                                                                                   
  def haversine(lat1, lon1, lat2, lon2):                                                                           
      R = 6371  # radius of the Earth in km                                                                        
      dlat = radians(lat2 - lat1)                                                                                  
      dlon = radians(lon2 - lon1)                                                                                  
      a = sin(dlat / 2)**2 + cos(radians(lat1)) * cos(radians(lat2)) * sin(dlon / 2)**2                            
      c = 2 * atan2(sqrt(a), sqrt(1 - a))                                                                          
      return R * c                                                                                                 
                                                                                                                   
  from datetime import timedelta                                                                                   
                                                                                                                   
  def calculate_cargo_travel_time(origin_coords, destination_coords):                                              
      cruising_speed_kmh = 750  # typical cargo plane speed in km/h                                                
      origin_lat, origin_lon = radians(origin_coords[0]), radians(origin_coords[1])                                
      destination_lat, destination_lon = radians(destination_coords[0]), radians(destination_coords[1])            
      distance_km [38;2;255;70;137;48;2;3

Execution logs:
Searching for Batman filming locations...
Searching for The Dark Knight filming locations...
Searching for Christopher Nolan filming locations...
Error checking #: You must install packages `markdownify` and `requests` to run this tool: for instance run `pip 
install markdownify requests`.
Error checking #: You must install packages `markdownify` and `requests` to run this tool: for instance run `pip 
install markdownify requests`.
Error checking  : You must install packages `markdownify` and `requests` to run this tool: for instance run `pip 
install markdownify requests`.
Error checking S: You must install packages `markdownify` and `requests` to run this tool: for instance run `pip 
install markdownify requests`.
Error checking e: You must install packages `markdownify` and `requests` to run this tool: for instance run `pip 
install markdownify requests`.
Error checking a: You must install packages `markdownify` and `requests` to run this tool: for instance run `pip 
install markdownify requests`.
Error checking r: You must install packages `markdownify` and `requests` to run this tool: for instance run `pip 
install markdownify requests`.
Error checking c: You must install packages `markdownify` and `requests` to run this tool: for instance run `pip 
install markdownify requests`.
Error checking h: You must install packages `markdownify` and `requests` to run this tool: for instance run `pip 
install markdownify requests`.
Error checking  : You must install packages `markdownify` and `requests` to run this tool: for instance run `pip 
install markdownify requests`.
Error checking R: You must install packages `markdownify` and `requests` to run this tool: for instance run `pip 
install markdownify requests`.
Error checking e: You must install packages `markdownify` and `requests` to run this tool: for instance run `pip 
install markdownify requests`.
Error checking s: You must install packages `markdownify` and `requests` to run this tool: for instance run `pip 
install markdownify requests`.
Error checking u: You must install packages `markdownify` and `requests` to run this tool: for instance run `pip 
install markdownify requests`.
Error checking l: You must install packages `markdownify` and `requests` to run this tool: for instance run `pip 
install markdownify requests`.
Error checking t: You must install packages `markdownify` and `requests` to run this tool: for instance run `pip 
install markdownify requests`.
Error checking s: You must install packages `markdownify` and `requests` to run this tool: for instance run `pip 
install markdownify requests`.
Error checking 
: You must install packages `markdownify` and `requests` to run this tool: for instance run `pip install 
markdownify requests`.
Error checking 0: You must install packages `markdownify` and `requests` to run this tool: for instance run `pip 
install markdownify requests`.
Error checking .: You must install packages `markdownify` and `requests` to run this tool: for instance run `pip 
install markdownify requests`.
Error checking  : You must install packages `markdownify` and `requests` to run this tool: for instance run `pip 
install markdownify requests`.
Error checking [: You must install packages `markdownify` and `requests` to run this tool: for instance run `pip 
install markdownify requests`.
Error checking T: You must install packages `markdownify` and `requests` to run this tool: for instance run `pip 
install markdownify requests`.
Error checking h: You must install packages `markdownify` and `requests` to run this tool: for instance run `pip 
install markdownify requests`.
Error checking e: You must install packages `markdownify` and `requests` to run this tool: for instance run `pip 
install markdownify requests`.
Error checking  : You must install packages `markdownify` and `requests` to run this tool: for instance run `pip 
install markdownify requests`.
Error checking B: You must install packages `markdownify` and `reque

Code execution failed at line 'supercar_factories_df = 
pd.DataFrame(find_supercar_factories(calculate_cargo_travel_time((40.7128, -74.0060), (37.7749, 
-122.4194)).total_seconds() / 3600))' due to: InterpreterError: Object 7.06 has no attribute total_seconds

[Step 5: Duration 64.13 seconds| Input tokens: 14,970 | Output tokens: 1,665]

KeyboardInterrupt: 

In [ ]:
model = InferenceClientModel(
    "Qwen/Qwen2.5-Coder-32B-Instruct", provider="together", max_tokens=8096
)

web_agent = CodeAgent(
    model=model,
    tools=[
        GoogleSearchTool(provider="serper"),
        VisitWebpageTool(),
        calculate_cargo_travel_time,
    ],
    name="web_agent",
    description="Browses the web to find information",
    verbosity_level=0,
    max_steps=10,
)

In [ ]:
from smolagents.utils import encode_image_base64, make_image_url
from smolagents import OpenAIServerModel


def check_reasoning_and_plot(final_answer, agent_memory):
    multimodal_model = OpenAIServerModel("gpt-4o", max_tokens=8096)
    filepath = "saved_map.png"
    assert os.path.exists(filepath), "Make sure to save the plot under saved_map.png!"
    image = Image.open(filepath)
    prompt = (
        f"Here is a user-given task and the agent steps: {agent_memory.get_succinct_steps()}. Now here is the plot that was made."
        "Please check that the reasoning process and plot are correct: do they correctly answer the given task?"
        "First list reasons why yes/no, then write your final decision: PASS in caps lock if it is satisfactory, FAIL if it is not."
        "Don't be harsh: if the plot mostly solves the task, it should pass."
        "To pass, a plot should be made using px.scatter_map and not any other method (scatter_map looks nicer)."
    )
    messages = [
        {
            "role": "user",
            "content": [
                {
                    "type": "text",
                    "text": prompt,
                },
                {
                    "type": "image_url",
                    "image_url": {"url": make_image_url(encode_image_base64(image))},
                },
            ],
        }
    ]
    output = multimodal_model(messages).content
    print("Feedback: ", output)
    if "FAIL" in output:
        raise Exception(output)
    return True


manager_agent = CodeAgent(
    model=InferenceClientModel("deepseek-ai/DeepSeek-R1", provider="together", max_tokens=8096),
    tools=[calculate_cargo_travel_time],
    managed_agents=[web_agent],
    additional_authorized_imports=[
        "geopandas",
        "plotly",
        "shapely",
        "json",
        "pandas",
        "numpy",
    ],
    planning_interval=5,
    verbosity_level=2,
    final_answer_checks=[check_reasoning_and_plot],
    max_steps=15,
)

In [ ]:
manager_agent.visualize()

In [ ]:
manager_agent.run("""
Find all Batman filming locations in the world, calculate the time to transfer via cargo plane to here (we're in Gotham, 40.7128° N, 74.0060° W).
Also give me some supercar factories with the same cargo plane transfer time. You need at least 6 points in total.
Represent this as spatial map of the world, with the locations represented as scatter points with a color that depends on the travel time, and save it to saved_map.png!

Here's an example of how to plot and return a map:
import plotly.express as px
df = px.data.carshare()
fig = px.scatter_map(df, lat="centroid_lat", lon="centroid_lon", text="name", color="peak_hour", size=100,
     color_continuous_scale=px.colors.sequential.Magma, size_max=15, zoom=1)
fig.show()
fig.write_image("saved_image.png")
final_answer(fig)

Never try to process strings using code: when you have a string to read, just print it and you'll see it.
""")

In [ ]:
manager_agent.python_executor.state["fig"]